In [ ]:
!pip install datasets transformers torch torchvision faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import torch
import faiss
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
from datetime import datetime

ModuleNotFoundError: No module named 'faiss'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Loaded Model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Loaded Model


In [ ]:
# Load reviews and metadata using Hugging Face datasets
#review_data = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)["full"]
product_meta_data = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True)

print(f"Loaded {len(product_meta_data)} products.")

#df_reviews = pd.DataFrame.from_records(review_data).add_prefix("review_")
df_meta    = pd.DataFrame.from_records(product_meta_data).add_prefix("product_")

#df_reviews.rename(columns={"review_parent_asin": "parent_asin"}, inplace=True)
df_meta.rename(columns={"product_parent_asin": "parent_asin"}, inplace=True)

merged_df = df_meta# pd.merge(df_reviews, df_meta, on="parent_asin", how="left")

def merge_text_data(row):
    parts = [
        row['product_title'],
        row['product_subtitle'],
        row['product_description'],
        row['product_details'],
        row['product_features'],
    ]
    # Keep only parts that are strings with non-whitespace characters.
    parts = [p for p in parts if isinstance(p, str) and p.strip()]
    return ". ".join(parts)

merged_df['Combined_text'] = merged_df.apply(merge_text_data, axis=1)
print("Combined text")


def extract_product_images(img_dict):
    urls = []
    if isinstance(img_dict, dict):
        for size in ("hi_res", "large", "thumb"):
            urls.extend(img_dict.get(size, []))
    return [u for u in urls if u and isinstance(u, str)]

merged_df["prod_imgs"] = merged_df["product_images"].apply(extract_product_images)
print("Extracted product images")

def download_and_filter_image(url):
    try:
        resp = requests.get(url, timeout=5)
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        return img
    except Exception:
        return None

def embed_text(text):
    inputs = clip_processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        emb = clip_model.get_text_features(**inputs)
    return (emb / emb.norm(dim=-1, keepdim=True))

def embed_images(urls):
    if not urls:
        return None

    # Download the first valid image from the list of URLs
    img = download_and_filter_image(urls[0])

    if img is not None:
        # Generate embeddings for the first image
        inputs = clip_processor(images=img, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            emb = clip_model.get_image_features(**inputs)
            emb = emb / emb.norm(dim=-1, keepdim=True)
        return emb
    else:
        return None

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

meta_All_Beauty.jsonl:   0%|          | 0.00/213M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/112590 [00:00<?, ? examples/s]

Loaded 112590 products.
Combined text
Extracted product images


In [ ]:
merged_df = merged_df[merged_df['product_title'].notna() & (merged_df['product_title'].str.strip() != '')]
merged_df.shape

(112578, 18)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
merged_df.to_csv('/content/drive/My Drive/Colab Notebooks/DL_2025/Project/beauty_products.csv', index=False)

In [ ]:
merged_df.columns

Index(['product_main_category', 'product_title', 'product_average_rating',
       'product_rating_number', 'product_features', 'product_description',
       'product_price', 'product_images', 'product_videos', 'product_store',
       'product_categories', 'product_details', 'parent_asin',
       'product_bought_together', 'product_subtitle', 'product_author',
       'Combined_text', 'prod_imgs'],
      dtype='object')

In [ ]:
merged_df.head(100)

,product_main_category,product_title,product_average_rating,product_rating_number,product_features,product_description,product_price,product_images,product_videos,product_store,product_categories,product_details,parent_asin,product_bought_together,product_subtitle,product_author,Combined_text,prod_imgs
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",[https://m.media-amazon.com/images/I/71i77AuI9...
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,[https://m.media-amazon.com/images/I/71g1lP0pM...
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None,Eye Patch Black Adult with Tie Band (6 Per Pac...,[https://m.media-amazon.com/images/I/31bz+uqzW...
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",[https://m.media-amazon.com/images/I/71GJhXQGv...
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None,Precision Plunger Bars for Cartridge Grips – 9...,[https://m.media-amazon.com/images/I/31TgqAZ8k...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,All Beauty,NLCAC Beaded Headband for Women Handmade Beade...,3.4,9,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",NLCAC,[],"{""Material"": ""Polyester, Cloth, Plastic, Cotto...",B08ND9SL4Q,None,None,None,NLCAC Beaded Headband for Women Handmade Beade...,[https://m.media-amazon.com/images/I/71RcyCZqS...
96,All Beauty,"Miss U Hair 20""/50cm 110g Long Curl/wavy Clip ...",4.6,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",miss u hair,[],"{""Brand"": ""Miss u hair"", ""Material"": ""Syntheti...",B010J7VQ88,None,None,None,"Miss U Hair 20""/50cm 110g Long Curl/wavy Clip ...",[https://m.media-amazon.com/images/I/619xY+bBQ...
97,All Beauty,African Pride Black Castor Miracle Hold and Co...,3.7,23,[],[African Pride Black Castor Miracle Hold & Cov...,9.99,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",African Pride,[],"{""Item Form"": ""Gel"", ""Brand"": ""African Pride"",...",B07N9715WM,None,None,None,African Pride Black Castor Miracle Hold and Co...,[https://m.media-amazon.com/images/I/51aAhYhmW...
98,All Beauty,Philips Norelco T970 Accu-Vac Beard and Mousta...,3.3,126,[Stainless Steel Self Sharpening Blades offer ...,"[Product Description, Norelco's AccuVac Beard ...",None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",PHILIPS,[],"{""Package Dimensions"": ""9.7 x 6 x 4.6 inches; ...",B0001HYJW2,None,None,None,Philips Norelco T970 Accu-Vac Beard and Mousta...,[https://m.media-amazon.com/images/I/31NWWEC1C...


In [ ]:
def build_records(df):
    records = []
    for _, row in df.iterrows():
        # Get text embedding for the combined text.
        text_emb = embed_text(row["Combined_text"])
        # Get image embeddings separately.
        img_emb = embed_images(row["prod_imgs"])
        # Combine text and image embeddings.
        if img_emb is not None:
            combined_emb = (text_emb + img_emb) / 2
        else:
            combined_emb = text_emb
        combined_emb = combined_emb / combined_emb.norm(dim=-1, keepdim=True)
        extra = {
              "product_title": row["product_title"],
              "product_price": row.get("product_price", "N/A"),
              "main_category": row.get("product_main_category", "N/A"),
              "prod_avg_rating": float(row.get("product_average_rating") or 0),
              "prod_rating_num": int(row.get("product_rating_number") or 0)
          }


        records.append({"emb": combined_emb.squeeze(0), "extra": extra})
    return records

df_sample = merged_df[:500].reset_index(drop=True)
records_sample = build_records(df_sample)
print(f"Built embeddings for {len(records_sample)} items.")


Built embeddings for 500 items.


In [ ]:
def query_to_embedding(query_text=None, query_image_paths=None):
    text_emb = None
    img_emb = None
    if query_text:
        text_emb = embed_text(query_text)
    if query_image_paths:
        img_emb = embed_images(query_image_paths)
    if text_emb is not None and img_emb is not None:
        combined_emb = (text_emb + img_emb) / 2
    elif text_emb is not None:
        combined_emb = text_emb
    elif img_emb is not None:
        combined_emb = img_emb
    else:
        raise ValueError("At least one of `query_text` or `query_image_paths` must be provided.")
    combined_emb = combined_emb / combined_emb.norm(dim=-1, keepdim=True)
    return combined_emb

def torch_similarity_search(query_emb, all_embs, top_k=5):
    sim_scores = torch.nn.functional.cosine_similarity(query_emb, all_embs)  # shape [N]
    top_scores, top_idxs = torch.topk(sim_scores, k=top_k)
    return top_idxs.tolist(), top_scores.tolist()

def recommend_from_query(query_text=None, query_image_paths=None, records=None, top_k=5, rerank_k=20):
    query_emb = query_to_embedding(query_text=query_text, query_image_paths=query_image_paths)
    all_embs = torch.stack([r["emb"] for r in records]).to(query_emb.device)  # [N,512]
    top_idxs, top_sims = torch_similarity_search(query_emb, all_embs, top_k=rerank_k)
    ranked = []
    for i, sim in zip(top_idxs, top_sims):
        ranked.append((records[i]["extra"]["product_title"],sim))
    ranked.sort(key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

In [ ]:
external_query_text = "vibrant, long-lasting, cruelty-free lipstick with natural ingredients"
recs_text = recommend_from_query(query_text=external_query_text, records=records_sample, top_k=5)
print("Recommendations for external text query:")
for title, sim in recs_text:
    print(f"→ {title[:60]:60s} | Similarity Score={sim:.3f}")

Recommendations for external text query:
→ Velvet Red Matte Lipstick, Red Rover, Vegan, Paraben-Free, C | Similarity Score=0.709
→ NYX Butter Lipstick LITTLE SUSIE                             | Similarity Score=0.664
→ [Bbia] Last Lip Mousse 3.7g / Soft Velvet Mousse Texture - 5 | Similarity Score=0.662
→ Jordana Matte Lipstick 48 Pink Passion                       | Similarity Score=0.659
→ OULAC Shine to Shine Velvet Matte Lipstick, Long Lasting Cre | Similarity Score=0.655


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DL_2025/Project/beauty_products.csv')

In [ ]:
df.columns

Index(['product_main_category', 'product_title', 'product_average_rating',
       'product_rating_number', 'product_features', 'product_description',
       'product_price', 'product_images', 'product_videos', 'product_store',
       'product_categories', 'product_details', 'parent_asin',
       'product_bought_together', 'product_subtitle', 'product_author',
       'Combined_text', 'prod_imgs'],
      dtype='object')

In [ ]:
# Randomly sample 50 entries from a specific column
sampled_entries = df['product_title'].sample(n=50, random_state=42)

# Optionally convert to a list
sampled_list = sampled_entries.tolist()

sampled_list

['8 Pieces Nose Rings Bone Stud, Nose Gem 20G comes with 1 Nose Retainer',
 'Vanity Case, Endless Love Lipstick, Corsica Lipshine',
 'Nail Files',
 'Bath and Body Works Vanilla Bean Noel Gift Set (Shower Gel, Body Lotion, and Body Splash) Holiday Traditions Collection 2008',
 'Leeons Scrunchy Updo Wavy Straight Hair Bun Messy Donut Elastic Chignons Wrap Around Synthetic Ponytail Hairpiece Hair Extension (#2/30)',
 'Vigorol Mousse Relaxed Hair 12 Ounce (354ml) (2 Pack)',
 'Mariska 10A Straight Human Hair Bundles with Closure Brazilian Hair Straight Bundles 100% Unprocessed Human Hair 3 Bundles with Closure for Black Women (16 18 20+14, Straight bundles with closure)',
 'Blitzby 20 Pcs Tattoo Lipstick Cigarette Cotton Swab Lip Stain, Portable Non-Stick Liquid Lipstick, Long-Lasting, and Waterproof Lip Tint. (Red)',
 'New Arrival! SOC Animal Cutie Moisturizing Facial Mask (4 Types x2)',
 'Cordless Water Flosser for Teeth Portable Dental Oral Irrigator Rechargeable with 4 Jet Tips, Teeth C

In [ ]:
sampled_entries.value_counts()

,count
product_title,
"8 Pieces Nose Rings Bone Stud, Nose Gem 20G comes with 1 Nose Retainer",1
"Vanity Case, Endless Love Lipstick, Corsica Lipshine",1
Nail Files,1
"Bath and Body Works Vanilla Bean Noel Gift Set (Shower Gel, Body Lotion, and Body Splash) Holiday Traditions Collection 2008",1
Leeons Scrunchy Updo Wavy Straight Hair Bun Messy Donut Elastic Chignons Wrap Around Synthetic Ponytail Hairpiece Hair Extension (#2/30),1
Vigorol Mousse Relaxed Hair 12 Ounce (354ml) (2 Pack),1
"Mariska 10A Straight Human Hair Bundles with Closure Brazilian Hair Straight Bundles 100% Unprocessed Human Hair 3 Bundles with Closure for Black Women (16 18 20+14, Straight bundles with closure)",1
"Blitzby 20 Pcs Tattoo Lipstick Cigarette Cotton Swab Lip Stain, Portable Non-Stick Liquid Lipstick, Long-Lasting, and Waterproof Lip Tint. (Red)",1
New Arrival! SOC Animal Cutie Moisturizing Facial Mask (4 Types x2),1
